# 서울시 CCTV 구별 단계구분도

In [1]:
import json, folium
import pandas as pd

- 데이터 준비하기

In [2]:
df = pd.read_csv('../02.Pandas/cctv_pop.csv', index_col='구별')
df.head(3)

,CCTV댓수,최근증가율,인구수,내국인,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,,
종로구,1715,20.69,153789,144683,9106,27818,5.92,18.09
중구,2447,78.98,131787,122499,9288,24392,7.05,18.51
용산구,2611,78.64,237285,222953,14332,39070,6.04,16.47


In [4]:
filename = 'skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding='utf-8'))

- 단계구분도(Choropleth)

In [6]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_str,
    data=df.CCTV댓수,
    columns=[df.index, df.CCTV댓수],
    fill_color='PuRd',
    key_on='feature.id'
).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">구별 CCTV 댓수</h3>'    
map.get_root().html.add_child(folium.Element(title_html)) 
map

- 구 이름 표시하기(좌표, 텍스트)

In [7]:
import numpy as np

def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['id']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

In [8]:
from folium.features import DivIcon

gu_dict = get_text_location(geo_str)

In [15]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_str,
    data=df.CCTV댓수,
    columns=[df.index, df.CCTV댓수],
    fill_color='YlGnBu',
    key_on='feature.id'
).add_to(map)
for gu_name in df.index:
    folium.Marker(
        location=gu_dict[gu_name],
        icon=DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">구별 CCTV 댓수</h3>'    
map.get_root().html.add_child(folium.Element(title_html)) 
map